In [10]:
pip install pandas requests langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=2c575bbfedcda50c3ecc611377b7565152d884cfb4503c6aa64373df93b23f44
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [12]:
pip install pandas requests beautifulsoup4


82f0ebb97fa645d0addb9ee2b0c13a14

In [21]:
import requests
import pandas as pd
from langdetect import detect, LangDetectException

def fetch_gdelt_articles(query, start_date, end_date):
    url = (
        f"https://api.gdeltproject.org/api/v2/doc/doc?"
        f"query={query}&mode=artlist&format=json&maxrecords=250"
        f"&startdatetime={start_date}000000&enddatetime={end_date}235959"
    )
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return []
    data = response.json()
    return data.get('articles', [])

def is_english(text):
    """Detect if the given text is in English."""
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

# === Configuration ===
QUERIES = ["Tesla", "Elon Musk"]
START_DATE = "20230101"  # January 1, 2023
END_DATE = "20231230"    # December 30, 2023

# Fetch articles and filter for English content
all_filtered_articles = []

for query in QUERIES:
    articles = fetch_gdelt_articles(query, START_DATE, END_DATE)

    filtered_articles = [
        (article['seendate'], article['title'], article['url'])
        for article in articles
        if is_english(article.get('title', ''))
    ]

    all_filtered_articles.extend(filtered_articles)

# Remove duplicates by converting to a set and back to a list
all_filtered_articles = list(set(all_filtered_articles))

# Save the filtered articles to a CSV file
if all_filtered_articles:
    df_articles = pd.DataFrame(all_filtered_articles, columns=['PublishedAt', 'Title', 'URL'])
    df_articles.to_csv('tesla_elon_english_articles.csv', index=False)
    print("English articles saved to 'tesla_elon_english_articles.csv'.")
else:
    print("No English articles found.")

English articles saved to 'tesla_elon_english_articles.csv'.


In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_article_content(url):
    """Fetch the content from a URL."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract article text - Adjust based on the website's HTML structure
        paragraphs = soup.find_all('p')
        article_content = ' '.join([p.get_text() for p in paragraphs])

        return article_content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""

# === Step 1: Read URLs from the CSV file ===
input_csv = 'tesla_elon_english_articles.csv'  # Replace with your CSV filename
df = pd.read_csv(input_csv)

# Ensure the 'URL' column exists
if 'URL' not in df.columns:
    raise ValueError("The CSV file must contain a 'URL' column.")

# === Step 2: Scrape Content from Each URL ===
scraped_data = []
for index, row in df.iterrows():
    url = row['URL']
    print(f"Scraping: {url}")
    content = scrape_article_content(url)
    scraped_data.append([row['PublishedAt'], row['Title'], url, content])

# === Step 3: Save the Scraped Data to a New CSV File ===
output_csv = 'scraped_articles.csv'
df_scraped = pd.DataFrame(scraped_data, columns=['PublishedAt', 'Title', 'URL', 'Content'])
df_scraped.to_csv(output_csv, index=False)

print(f"Scraped data saved to '{output_csv}'.")


Scraping: https://www.ndtv.com/offbeat/almost-as-good-as-you-elon-musks-reply-to-mother-on-old-halloween-pic-4530673
Scraping: https://www.usmagazine.com/celebrity-news/news/grimes-sues-ex-elon-musk-over-parental-rights-of-their-children/
Scraping: https://www.myrtlebeachonline.com/news/politics-government/article280883988.html
Error fetching https://www.myrtlebeachonline.com/news/politics-government/article280883988.html: HTTPSConnectionPool(host='www.myrtlebeachonline.com', port=443): Read timed out. (read timeout=10)
Scraping: https://www.breitbart.com:443/economy/2023/11/06/elon-musks-x-twitter-starts-selling-account-handles-for-up-to-50000/
Scraping: https://www.local10.com/business/2023/10/02/tesla-3q-sales-rise-27-but-fall-short-of-analysts-estimates-due-to-price-cuts-and-factory-downtime/
Scraping: https://www.aceshowbiz.com/news/view/00213855.html
Error fetching https://www.aceshowbiz.com/news/view/00213855.html: 403 Client Error: Forbidden for url: https://www.aceshowbiz.com/